# 1. 박스오피스 데이터 전처리

1. kofic 일별 박스오피스 데이터 (최종_박스오피스_데이터.csv)
2. cgv 줄거리 파일 (cgv_장르_줄거리_통합본.csv) - 기준
분석에 사용할 박스오피스 데이터만 걸러낸다

## 1) 데이터 필터

In [1]:
import pandas as pd
import numpy as np

In [2]:
kofic_data=pd.read_csv("../영화관람객크롤링/최종_박스오피스_데이터.csv",encoding='utf-8-sig')
cgv_data=pd.read_csv("../영화리뷰크롤링/cgv리뷰_전처리/전처리파일저장/cgv_장르_줄거리_통합본.csv",encoding='utf-8-sig')

In [3]:
kofic_data.head(1)

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분
0,#Take No. 고백(인디스데이),NaN,13000,2,1,1,한국,NaN,NaN,기타,NaN,NaN,2023-02-10,최초 개봉


In [4]:
cgv_data.head(1)

,영화제목,장르,줄거리
0,좋은날,애니메이션,"20대의 풋풋한 사랑 [봄봄], 40대의 처참했던 슬픔 [운수 좋은 날] 그리고 6..."


데이터 중복 확인

In [5]:
kofic_data[["영화명","일자"]].duplicated().sum()

np.int64(1894)

중복 데이터 1894개 제거

In [6]:
kofic_data.drop_duplicates(subset=["영화명","일자"],inplace=True)

## 2) 사용할 영화

In [7]:
barometer=cgv_data["영화제목"]
temp=kofic_data[kofic_data["영화명"].isin(barometer)]

In [8]:
temp["영화명"].nunique()

6472

## 3) 최초개봉만

In [9]:
temp=temp[temp["개봉 구분"]=="최초 개봉"]
temp.head()

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분
12,#살아있다,2020-06-24,994000,142,1,1,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-02,최초 개봉
13,#살아있다,2020-06-24,909000,449,2,2,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-03,최초 개봉
14,#살아있다,2020-06-24,7314000,1386,7,11,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-15,최초 개봉
15,#살아있다,2020-06-24,3950000,790,6,6,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-19,최초 개봉
16,#살아있다,2020-06-24,2503000,1239,5,5,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-23,최초 개봉


## 4) n주차 생성

개봉일이 없는 영화 - 그 영화는 모든 일자에 개봉일이 없나?  
개봉일이 하나도 없는 영화는 제거

In [10]:
temp = temp[temp.groupby('영화명')['개봉일'].transform('count') > 0]

In [11]:
temp.isna().sum()

영화명          0
개봉일         86
매출액          0
관객수          0
스크린수         0
상영횟수         0
대표국적         0
배급사       1254
등급         186
장르         143
감독        7709
배우       16323
일자           0
개봉 구분        0
dtype: int64

개봉일이 있는것과 없는 영화도 있음  
결측치 보간

In [12]:
temp['개봉일'] = temp.groupby('영화명')['개봉일'].transform('first')

In [13]:
temp.isna().sum()

영화명          0
개봉일          0
매출액          0
관객수          0
스크린수         0
상영횟수         0
대표국적         0
배급사       1254
등급         186
장르         143
감독        7709
배우       16323
일자           0
개봉 구분        0
dtype: int64

날짜 형식 변경  
  
1주차: 영화 개봉일이 포함된 주의 일요일까지  
2주차 이후: 월요일부터 일요일까지를 한 주로 계산

In [14]:
temp['개봉일_dt'] = pd.to_datetime(temp['개봉일'], errors='coerce')
temp['일자_dt'] = pd.to_datetime(temp['일자'], errors='coerce')

In [15]:
temp.head()

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분,개봉일_dt,일자_dt
12,#살아있다,2020-06-24,994000,142,1,1,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-02,최초 개봉,2020-06-24,2020-06-02
13,#살아있다,2020-06-24,909000,449,2,2,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-03,최초 개봉,2020-06-24,2020-06-03
14,#살아있다,2020-06-24,7314000,1386,7,11,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-15,최초 개봉,2020-06-24,2020-06-15
15,#살아있다,2020-06-24,3950000,790,6,6,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-19,최초 개봉,2020-06-24,2020-06-19
16,#살아있다,2020-06-24,2503000,1239,5,5,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-23,최초 개봉,2020-06-24,2020-06-23


일자가 개봉일보다 빠른 것은 제거 (개봉전 시사회일 가능성 높음)

In [16]:
temp=temp[temp['개봉일_dt'] <= temp['일자_dt']]

각 영화별로 고유한 첫 개봉일을 '개봉일_대표' 컬럼으로 지정 (날짜 형식 변경한 것을 사용)

In [17]:
temp['개봉일_대표'] = temp.groupby('영화명')['개봉일_dt'].transform('first')

개봉일이 속한 주의 일요일을 '첫주_마감일'로 계산  
월요일 : 0, 일요일 : 6

In [18]:
days_to_sunday = 6 - temp['개봉일_대표'].dt.weekday
temp['첫주_마감일'] = temp['개봉일_대표'] + pd.to_timedelta(days_to_sunday, unit='D')

주차 계산

In [19]:
# 일자_dt가 첫주_마감일보다 작거나 같으면 1일차
is_first_week = temp['일자_dt'] <= temp['첫주_마감일']
# 일자_dt와 첫주_마감일의 차이를 일수로 계산
days_diff = (temp['일자_dt'] - temp['첫주_마감일']).dt.days
# 2주차 부터의 주차 계산
weeks_after_first: int = (days_diff - 1) // 7 + 2 # 값 자체가 주차 숫자
# 첫주차면 1, 아니면 계산된 주차 숫자를 입력
temp['주차'] = np.where(is_first_week, 1, weeks_after_first)

이제 필요없는 컬럼 삭제

In [20]:
temp.drop(columns=['개봉일_dt', '일자_dt', '개봉일_대표', '첫주_마감일'], inplace=True)

확인

In [21]:
temp.tail(5)

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분,주차
682460,힛쳐,2007-08-01,54000,8,2,3,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-08,최초 개봉,2
682461,힛쳐,2007-08-01,54000,8,1,2,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-10,최초 개봉,2
682462,힛쳐,2007-08-01,26000,4,1,2,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-11,최초 개봉,2
682463,힛쳐,2007-08-01,60000,9,1,2,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-12,최초 개봉,2
682464,힛쳐,2007-08-01,7000,1,1,1,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-13,최초 개봉,3


In [22]:
temp[temp["영화명"]=="#살아있다"].groupby("주차").size()

주차
1     5
2     7
3     7
4     7
5     7
6     7
7     7
8     7
9     5
10    3
19    1
dtype: int64

In [23]:
pd.set_option('display.max_rows', 100)
check_temp = temp.groupby(['영화명', '주차']).size().reset_index(name='counts')
check_temp.head(100)

,영화명,주차,counts
0,#살아있다,1,5
1,#살아있다,2,7
2,#살아있다,3,7
3,#살아있다,4,7
4,#살아있다,5,7
5,#살아있다,6,7
6,#살아있다,7,7
7,#살아있다,8,7
8,#살아있다,9,5
9,#살아있다,10,3


In [24]:
pd.set_option('display.max_rows', 10)

각 주차별 일자가 7일이 넘는지 확인 (데이터 중복 검사)

In [25]:
check_temp[check_temp['counts'] > 7]

,영화명,주차,counts


문제점  
1. 주차가 확 띄는 경우가 있다 -> 재개봉이 잘 걸러지지 않았다, 따라서 이를 제거  
2. 중간중간 데이터가 비는 경우가 있다 -> 영화별로 얼만큼 비는지 확인

재개봉 주차를 제거  
마지막 개봉주차와의 차이가 1보다 크면 재개봉이라고 판단

In [26]:
check_temp['주차_차이'] = check_temp.groupby('영화명')['주차'].diff()

1~7, 13, 14주차와 같은 경우 14주차는 분명 재개봉이지만 주차 차이는 1이다  
이것은 누적합으로 해결한다

이 방법대로 하면 1~8, 10~주차의 경우 재개봉이 아님에도 판별 못할 수 있다  
그러나 이정도 차이가 나면 결측치 보간을 할때 그 정확도가 매우 떨어진다  
따라서 해당 영화를 아예 제거하는 것보다, 일부분만이라도 사용하는게 나을 수 있지 않을까?  
(어차피 인기있는 영화면 박스오피스 데이터가 누락될 일이 없을 것)

In [27]:
check_temp["재개봉_시작_주차"] = check_temp.groupby('영화명')['주차'].diff()>1 #재개봉 시작 주차 표시
check_temp["재개봉_누적"] = check_temp.groupby("영화명")["재개봉_시작_주차"].cumsum()


In [28]:
pd.set_option('display.max_rows', 1000)
check_temp.head(1000)

,영화명,주차,counts,주차_차이,재개봉_시작_주차,재개봉_누적
0,#살아있다,1,5,NaN,False,0
1,#살아있다,2,7,1.0,False,0
2,#살아있다,3,7,1.0,False,0
3,#살아있다,4,7,1.0,False,0
4,#살아있다,5,7,1.0,False,0
5,#살아있다,6,7,1.0,False,0
6,#살아있다,7,7,1.0,False,0
7,#살아있다,8,7,1.0,False,0
8,#살아있다,9,5,1.0,False,0
9,#살아있다,10,3,1.0,False,0


In [29]:
pd.set_option('display.max_rows', 10)

In [30]:
check_temp=check_temp[check_temp['재개봉_누적'] == 0]

본 분석은 2주차 까지의 데이터를 사용한다.  
반대로 말하면, 2주차 이후의 데이터가 없으면 해당 영화는 사용할 수 없다

In [31]:
check_temp.shape

(37771, 6)

In [32]:
# '주차' 열에 2가 포함된 영화명을 추출합니다.
movies_with_week_2 = check_temp[check_temp['주차'] == 2]['영화명'].unique()

# 전체 영화명 목록을 가져옵니다.
all_movies = check_temp['영화명'].unique()

# 전체 영화 목록에서 2주차 데이터가 있는 영화 목록을 제외합니다.
movies_without_week_2 = [movie for movie in all_movies if movie not in movies_with_week_2]

# 최종 개수를 출력합니다.
print(len(movies_without_week_2))

492


In [33]:
pd.set_option('display.max_rows', 100)
wk_2 = check_temp[check_temp["영화명"].isin(movies_without_week_2)].groupby(["영화명", "주차"]).size().reset_index(name='counts')
wk_2.head(100)

,영화명,주차,counts
0,2024 태민 월드 투어 [이페머럴 게이즈],1,1
1,28일 후,826,1
2,28일 후,827,1
3,28일 후,828,1
4,28일 후,829,1
5,3670,1,1
6,4인용 식탁,346,1
7,7인의 황후,1,1
8,8 마일,40,1
9,8 마일,41,1


확인 결과 : 최초개봉일과 일자 차이가 많다 -> 박스오피스 계측 전(2003년 11월) 이전의 영화의 관람객 데이터는 없다. 이것이 후에 재개봉하면 자연스레 주차수가 비정상적으로 커진다

In [34]:
check_temp = check_temp[~check_temp["영화명"].isin(movies_without_week_2)]
check_temp.head(20)

,영화명,주차,counts,주차_차이,재개봉_시작_주차,재개봉_누적
0,#살아있다,1,5,NaN,False,0
1,#살아있다,2,7,1.0,False,0
2,#살아있다,3,7,1.0,False,0
3,#살아있다,4,7,1.0,False,0
4,#살아있다,5,7,1.0,False,0
5,#살아있다,6,7,1.0,False,0
6,#살아있다,7,7,1.0,False,0
7,#살아있다,8,7,1.0,False,0
8,#살아있다,9,5,1.0,False,0
9,#살아있다,10,3,1.0,False,0


In [35]:
pd.set_option('display.max_rows', 10)

## 5) 주차관련 오류 최종 수정

이제 해당 영화명과 주차를 가지고 temp에서 데이터를 최종 필터링한다

In [36]:
result = pd.merge(temp, check_temp[["영화명","주차"]], how="inner")
result.head(10)

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분,주차
0,#살아있다,2020-06-24,1311296000,204066,1662,7223,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-24,최초 개봉,1
1,#살아있다,2020-06-24,1240607900,148991,1637,7311,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-25,최초 개봉,1
2,#살아있다,2020-06-24,1380200100,152550,1713,7839,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-26,최초 개봉,1
3,#살아있다,2020-06-24,2644122800,294255,1882,9221,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-27,최초 개봉,1
4,#살아있다,2020-06-24,2263447540,256133,1869,9044,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-28,최초 개봉,1
5,#살아있다,2020-06-24,556892400,67845,1644,7111,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-29,최초 개봉,2
6,#살아있다,2020-06-24,543848720,67098,1641,7147,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-30,최초 개봉,2
7,#살아있다,2020-06-24,428624500,51659,1441,6137,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-07-01,최초 개봉,2
8,#살아있다,2020-06-24,368804680,44470,1401,5783,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-07-02,최초 개봉,2
9,#살아있다,2020-06-24,467173540,51860,1444,6066,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-07-03,최초 개봉,2


In [37]:
pd.set_option('display.max_rows', 100)
result.groupby(["영화명","주차"]).size().reset_index(name='counts').head(100)

,영화명,주차,counts
0,#살아있다,1,5
1,#살아있다,2,7
2,#살아있다,3,7
3,#살아있다,4,7
4,#살아있다,5,7
5,#살아있다,6,7
6,#살아있다,7,7
7,#살아있다,8,7
8,#살아있다,9,5
9,#살아있다,10,3


In [38]:
pd.set_option('display.max_rows', 10)

In [39]:
result["영화명"].nunique()

4773

최종 4773개의 영화가 추출되었다

## 6) 공휴일 등 날짜 결합

공휴일과 날짜가 있는 데이터를 일자를 기준으로 결합한다

In [40]:
date_data = pd.read_csv("../calendar_dataset.csv", encoding='utf-8-sig')
date_data.head()

,Date,Holiday,Moonhwa
0,2003-01-01,1,0
1,2003-01-02,0,0
2,2003-01-03,0,0
3,2003-01-04,1,0
4,2003-01-05,1,0


In [41]:
date_data.isna().sum() 

Date       0
Holiday    0
Moonhwa    0
dtype: int64

In [42]:
date_data["Date"] = pd.to_datetime(date_data["Date"], errors='coerce')
date_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8340 entries, 0 to 8339
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     8340 non-null   datetime64[ns]
 1   Holiday  8340 non-null   int64         
 2   Moonhwa  8340 non-null   int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 195.6 KB


In [43]:
result["일자"] = pd.to_datetime(result["일자"], errors='coerce')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197697 entries, 0 to 197696
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   영화명     197697 non-null  object        
 1   개봉일     197697 non-null  object        
 2   매출액     197697 non-null  int64         
 3   관객수     197697 non-null  int64         
 4   스크린수    197697 non-null  int64         
 5   상영횟수    197697 non-null  int64         
 6   대표국적    197697 non-null  object        
 7   배급사     197322 non-null  object        
 8   등급      197631 non-null  object        
 9   장르      197578 non-null  object        
 10  감독      191135 non-null  object        
 11  배우      184300 non-null  object        
 12  일자      197697 non-null  datetime64[ns]
 13  개봉 구분   197697 non-null  object        
 14  주차      197697 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(9)
memory usage: 22.6+ MB


일자 기준으로 병합

In [44]:
result.shape

(197697, 15)

In [45]:
temp_result = pd.merge(result, date_data, left_on = "일자", right_on = "Date", how = "left")
temp_result.shape


(197697, 18)

In [46]:
temp_result.drop(columns=["일자"], inplace=True)
temp_result.head(2)

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,개봉 구분,주차,Date,Holiday,Moonhwa
0,#살아있다,2020-06-24,1311296000,204066,1662,7223,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",최초 개봉,1,2020-06-24,0,1
1,#살아있다,2020-06-24,1240607900,148991,1637,7311,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",최초 개봉,1,2020-06-25,0,0


`펜데믹 구분` 은 아래와 같은 규칙으로 생성한다

1. 2020년 1월 까지는 `0`
2. 2020년 2월부터 2023년 4월까지는 `1`
3. 2023년 5월부터는 `2`

In [47]:
def get_pandemic_flag(date):
    if date < pd.Timestamp('2020-02-01'):
        return 0
    elif date < pd.Timestamp('2023-05-01'):
        return 1
    else:
        return 2

temp_result['Pandemic'] = temp_result['Date'].apply(get_pandemic_flag)

## 7) 최종 사용 컬럼 및 영문명 변경

In [48]:
final_result = \
temp_result.rename(columns={
    "영화명": "Movie_Title",
    "개봉일" : "Release_Date",
    "관객수": "Audience_Count",
    "상영횟수":"Show_Count",
    "대표국적":"Main_Country",
    "배급사":"Distributor",
    "등급":"Grade",
    "장르":"Genre",
    "주차":"Week"
    })

In [49]:
final_result.drop(columns=["매출액", "스크린수", "감독", "배우", "개봉 구분"], inplace=True)

## 8) 9월 6일 기준 개봉중인 영화

데이터는 9월 6일까지를 추출하였다  
모델 학습 시 상영이 끝난 영화만 사용해야 하기 때문에 이것들을 제거하여야 한다

9월 6일 기준 상영중인 영화면 `now_showing`열이 1이다

In [50]:
now_movie_list=final_result[final_result["Date"]=="2025-09-06"]["Movie_Title"].unique()
final_result["now_showing"] = final_result["Movie_Title"].apply(lambda x: 1 if x in now_movie_list else 0)

In [51]:
final_result[final_result["now_showing"]==1]

,Movie_Title,Release_Date,Audience_Count,Show_Count,Main_Country,Distributor,Grade,Genre,Week,Date,Holiday,Moonhwa,Pandemic,now_showing
3965,F1 더 무비,2025-06-25,82917,5279,미국,워너브러더스 코리아(주),12세이상관람가,"드라마,액션",1,2025-06-25,0,1,2,1
3966,F1 더 무비,2025-06-25,47737,4890,미국,워너브러더스 코리아(주),12세이상관람가,"드라마,액션",1,2025-06-26,0,0,2,1
3967,F1 더 무비,2025-06-25,65986,5212,미국,워너브러더스 코리아(주),12세이상관람가,"드라마,액션",1,2025-06-27,0,0,2,1
3968,F1 더 무비,2025-06-25,146966,5427,미국,워너브러더스 코리아(주),12세이상관람가,"드라마,액션",1,2025-06-28,1,0,2,1
3969,F1 더 무비,2025-06-25,134479,5099,미국,워너브러더스 코리아(주),12세이상관람가,"드라마,액션",1,2025-06-29,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185122,프리키 프라이데이 2,2025-08-27,2366,444,미국,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,"코미디,가족,판타지",2,2025-09-02,0,0,2,1
185123,프리키 프라이데이 2,2025-08-27,536,109,미국,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,"코미디,가족,판타지",2,2025-09-03,0,0,2,1
185124,프리키 프라이데이 2,2025-08-27,822,116,미국,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,"코미디,가족,판타지",2,2025-09-04,0,0,2,1
185125,프리키 프라이데이 2,2025-08-27,650,84,미국,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,"코미디,가족,판타지",2,2025-09-05,0,0,2,1


# 2. 박스오피스와 영화정보 데이터 분할

- 박스오피스 데이터는 영화명과 관객수, 상영횟수, 일자와 같은 상영정보만 모은다
- 영화 줄거리나 장르, 배급사 등은 영화 정보 데이터로 분할한다

In [52]:
final_result.head(1)

,Movie_Title,Release_Date,Audience_Count,Show_Count,Main_Country,Distributor,Grade,Genre,Week,Date,Holiday,Moonhwa,Pandemic,now_showing
0,#살아있다,2020-06-24,204066,7223,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,1,2020-06-24,0,1,1,0


In [53]:
final_result.columns

Index(['Movie_Title', 'Release_Date', 'Audience_Count', 'Show_Count',
       'Main_Country', 'Distributor', 'Grade', 'Genre', 'Week', 'Date',
       'Holiday', 'Moonhwa', 'Pandemic', 'now_showing'],
      dtype='object')

In [54]:
box_office = final_result[['Movie_Title', 'Release_Date', 'Audience_Count', 'Show_Count','Week', 'Date',
       'Holiday', 'Moonhwa', 'Pandemic', 'now_showing']].copy()
movie_info = final_result[['Movie_Title', 'Main_Country', 'Distributor', 'Grade', 'Genre', 'now_showing']].copy()

In [55]:
movie_info = movie_info.drop_duplicates()
movie_info["Movie_Title"].value_counts()

Movie_Title
카니발        2
10년        2
트럭         2
소풍         2
오랜만이다      1
          ..
못말리는 결혼    1
못          1
몰리스 게임     1
몬테크리스토     1
힛쳐         1
Name: count, Length: 4773, dtype: int64

In [56]:
movie_info[movie_info["Movie_Title"].duplicated(keep=False)]

,Movie_Title,Main_Country,Distributor,Grade,Genre,now_showing
460,10년,일본,(주)디오시네마,전체관람가,드라마,0
474,10년,홍콩,NaN,NaN,드라마,0
100204,소풍,한국,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,드라마,0
100381,소풍,홍콩,NaN,NaN,다큐멘터리,0
163205,카니발,스페인,씨네그루(주)다우기술,청소년관람불가,스릴러,0
163206,카니발,포르투갈,NaN,NaN,기타,0
174935,트럭,한국,(주)싸이더스,청소년관람불가,"범죄,스릴러",0
174977,트럭,프랑스,NaN,NaN,드라마,0


후에 cgv데이터과 결합 시 영화 제목이 중복되면 병합을 할 수 없다. 따라서 해당 영화는 제거하자 (4개밖에 안된다)

In [57]:
delete_movie = movie_info[movie_info["Movie_Title"].duplicated(keep=False)]["Movie_Title"].unique()
box_office = box_office[~box_office["Movie_Title"].isin(delete_movie)]
movie_info = movie_info[~movie_info["Movie_Title"].isin(delete_movie)]

In [58]:
box_office.reset_index(drop=True, inplace=True)
movie_info.reset_index(drop=True, inplace=True)

# 3. cgv 데이터

cgv에서 추출한 배우 목록과 장르, 줄거리 데이터를 가져오고 영화 정보 데이터에 병합한다  
이때 웹크롤링 과정 중 영화 중복 오류가 있기 때문이 이것을 최종 확인한 후 박스오피스 데이터를 수정 및 저장한다

In [59]:
cgv_actor = pd.read_csv("../영화리뷰크롤링/cgv리뷰_전처리/전처리파일저장/cgv_전처리_통합파일.csv", encoding='utf-8-sig')
cgv_info = pd.read_csv("../영화리뷰크롤링/cgv리뷰_전처리/전처리파일저장/cgv_장르_줄거리_통합본.csv", encoding='utf-8-sig')

In [60]:
cgv_actor.head()

,영화제목,감독1,감독2,감독3,감독4,감독5,배우1,배우2,배우3,배우4,배우5,배우6
0,매트릭스3 레볼루션,앤디 워쇼스키,라나 워쇼스키,NaN,NaN,NaN,키아누 리브스,모니카 벨루치,로렌스 피쉬번,NaN,NaN,NaN
1,써클,박기형,NaN,NaN,NaN,NaN,김혜성,이행석,이태성,장희진,NaN,NaN
2,폴리와 함께,존 햄버거,NaN,NaN,NaN,NaN,필립 세이무어 호프만,데브라 메싱,제니퍼 애니스톤,알렉 볼드윈,NaN,NaN
3,시월애,이현승,NaN,NaN,NaN,NaN,이인철,전지현,이정재,김무생,NaN,NaN
4,보리밭을 흔드는 바람,켄 로치,NaN,NaN,NaN,NaN,킬리언 머피,NaN,NaN,NaN,NaN,NaN


In [61]:
cgv_info.head()

,영화제목,장르,줄거리
0,좋은날,애니메이션,"20대의 풋풋한 사랑 [봄봄], 40대의 처참했던 슬픔 [운수 좋은 날] 그리고 6..."
1,황색 눈물,드라마,"1963년, 꿈을 쫒는 청춘의 눈부신 날들 1963년(쇼와38년) 늦은 봄, ..."
2,렉토베르소,드라마,"5명의 여성감독이 그려낸 사랑, 운명, 짝사랑, 후회에 관한 5편의 옴니버스 영화...."
3,"줄리엣, 네이키드",멜로/로맨스,"25년 전 앨범을 내고 홀연히 사라진 싱어송라이터, 터커 크로우(에단 호크). 애니..."
4,황후화,액션,"꽃으로 물든, 황금빛 반란이 시작된다! 중국 중세를 배경으로 하는 시대물로 궁궐 ..."


## 1) 영화 중복 검사

In [62]:
cgv_actor["영화제목"].value_counts()

영화제목
매트릭스3 레볼루션                     1
실버맨                            1
킴스 비디오                         1
써로게이트                          1
씨앗의 시간                         1
                              ..
호박과 마요네즈                       1
기동전사 건담 디 오리진 III - 새벽의 봉기     1
기동전사 건담 디 오리진 I - 푸른 눈의 캬스발    1
기동전사 건담 디 오리진 IV - 운명의 전야      1
톤레삽강은 멈추지 않는다                  1
Name: count, Length: 7001, dtype: int64

In [63]:
cgv_info["영화제목"].value_counts()

영화제목
좋은날          1
위 아 엑스       1
대전 블루스       1
럭키 몬스터       1
오마주          1
            ..
나의 특별한 형제    1
페르소나         1
미녀는 괴로워      1
방문객          1
암살자들         1
Name: count, Length: 6473, dtype: int64

중복 문제는 없다

## 2) 병합

In [64]:
cgv_merge = pd.merge(cgv_actor, cgv_info, on="영화제목", how="inner")
cgv_merge.head()

,영화제목,감독1,감독2,감독3,감독4,감독5,배우1,배우2,배우3,배우4,배우5,배우6,장르,줄거리
0,매트릭스3 레볼루션,앤디 워쇼스키,라나 워쇼스키,NaN,NaN,NaN,키아누 리브스,모니카 벨루치,로렌스 피쉬번,NaN,NaN,NaN,NaN,"감히 상상도 못할 결말,시작이 있는 곳에 끝이 있다! 2199년, 시스템이 인간을 ..."
1,써클,박기형,NaN,NaN,NaN,NaN,김혜성,이행석,이태성,장희진,NaN,NaN,액션,"열정, 우정, 사랑을 지키고 싶던 그들. 청춘의 피 끓는 열정이 폭력을 만나다."
2,시월애,이현승,NaN,NaN,NaN,NaN,이인철,전지현,이정재,김무생,NaN,NaN,드라마,이 사랑이 떠나지 않게 해주세요 사랑이었다는 걸 너무 늦게 알았습니다 1998년 1...
3,파이란,송해성,NaN,NaN,NaN,NaN,최민식,김지영,손병호,공형진,NaN,NaN,드라마,어디에서도 인정받지 못하는 삼류 건달 ‘강재’는 막장 인생의 마지막 찬스로 평생 꿈...
4,쓰레기,모함마드 아흐마디,NaN,NaN,NaN,NaN,모하마드 에스칸다리,파르진 모하데스,NaN,NaN,NaN,NaN,드라마,"낙엽이 지는 테헤란, 시인을 꿈꾸는 청소부!!!"


In [65]:
cgv_merge.rename(columns={
    '영화제목': 'Movie_Title',
    '감독1': 'director1',
    '감독2': 'director2',
    '감독3': 'director3',
    '감독4': 'director4',
    '감독5': 'director5',
    '배우1': 'actor1',
    '배우2': 'actor2',
    '배우3': 'actor3',
    '배우4': 'actor4',
    '배우5': 'actor5',
    '배우6': 'actor6',
    '장르': 'cgv_genre',
    '줄거리': 'movie_summary'
}, inplace=True)

In [66]:
movie_info=movie_info.merge(cgv_merge, on="Movie_Title", how="left")

In [67]:
movie_info.head()

,Movie_Title,Main_Country,Distributor,Grade,Genre,now_showing,director1,director2,director3,director4,director5,actor1,actor2,actor3,actor4,actor5,actor6,cgv_genre,movie_summary
0,#살아있다,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,0,조일형,NaN,NaN,NaN,NaN,박신혜,유아인,NaN,NaN,NaN,NaN,드라마,원인불명 증세의 사람들의 공격에 통제 불능에 빠진 도시. 영문도 모른 채 잠에서 ...
1,#아이엠히어,프랑스,(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,"멜로/로맨스,코미디",0,에릭 라티고,NaN,NaN,NaN,NaN,빈센트 네메스,알랭 샤바,카밀 러더퍼드,배두나,블랑슈 가르댕,NaN,"코미디, 멜로/로맨스",無대포! 無대책! 無눈치! 프렌치 러버의 좌충우돌 힐링 여행! “나도 지금 전성기...
2,#위왓치유,기타,찬란,청소년관람불가,다큐멘터리,0,바르보라 차르포바,비트 클루삭,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,다큐멘터리,"평범한 집처럼 꾸며진 3개의 세트장, 12살로 설정한 페이크 계정을 만들고 컴퓨터 ..."
3,(여자)아이들 월드투어 [아이돌] 인 시네마,한국,"씨제이포디플렉스 주식회사,씨지브이 아이스콘",12세이상관람가,공연,0,오윤동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공연,“진짜 제대로 즐길 준비 됐죠?” 2024년 전 세계 14개 도시의 관객을 ...
4,0.0MHz,한국,(주)스마일이엔티,15세이상관람가,공포(호러),0,유선동,NaN,NaN,NaN,NaN,박명신,강태웅,정원창,정은지,NaN,NaN,공포(호러),"에디슨의 마지막 발명품, 유령 탐지기? “증명할 수 없는 미스터리란 없다” 가지 말..."


## 3) 결측치 검사

In [68]:
movie_info.columns

Index(['Movie_Title', 'Main_Country', 'Distributor', 'Grade', 'Genre',
       'now_showing', 'director1', 'director2', 'director3', 'director4',
       'director5', 'actor1', 'actor2', 'actor3', 'actor4', 'actor5', 'actor6',
       'cgv_genre', 'movie_summary'],
      dtype='object')

In [69]:
movie_info[['Movie_Title', 'Main_Country', 'Distributor', 'Grade', 'Genre',
       'now_showing','cgv_genre', 'movie_summary', 'director1', 'actor1']].isna().sum()

Movie_Title        0
Main_Country       0
Distributor       20
Grade              9
Genre              6
now_showing        0
cgv_genre         49
movie_summary      0
director1        225
actor1           239
dtype: int64

### 문제점1 : 배급사(distributor), 등급(grade)가 없는것은 그냥 제거 -> 예

In [70]:
movie_info.dropna(subset=['Distributor', 'Grade'], inplace=True)

#### 등급 처리

In [71]:
movie_info["Grade"].unique()

array(['15세이상관람가', '12세이상관람가', '청소년관람불가', '전체관람가', '15세관람가',
       '18세관람가,청소년관람불가', '18세관람가', '12세관람가', '청소년관람불가,15세이상관람가',
       '15세관람가,15세이상관람가', '12세이상관람가,전체관람가', '12세이상관람가,12세관람가',
       '18세관람가,15세이상관람가', '15세관람가,12세이상관람가', '제한상영가'], dtype=object)

등급을 전체, 12세 이상, 15세 이상, 청소년관람불가 네가지로 재분류한다  
- 원래는 18세였지만 2024년 5월 1일 연령 기준 변경에 맞춰 19세로 개정

In [72]:
movie_info=movie_info[movie_info["Grade"]!='제한상영가']
movie_info.loc[:, "Grade"] = movie_info["Grade"].replace({
    '15세관람가' : '15세이상관람가',
    '18세관람가,청소년관람불가' : '청소년관람불가', 
    '18세관람가' : '청소년관람불가', 
    '12세관람가' : '12세이상관람가', 
    '청소년관람불가,15세이상관람가' : '15세이상관람가',
    '15세관람가,15세이상관람가' : '15세이상관람가', 
    '12세이상관람가,전체관람가' : '전체관람가', 
    '12세이상관람가,12세관람가' : '12세이상관람가',
    '18세관람가,15세이상관람가' : '15세이상관람가', 
    '15세관람가,12세이상관람가' : '12세이상관람가',
    '청소년관람불가' : '청소년관람불가',
    '18세이상관람가' : '청소년관람불가',
})

In [73]:
movie_info["Grade"].unique()

array(['15세이상관람가', '12세이상관람가', '청소년관람불가', '전체관람가'], dtype=object)

#### 배급사 처리

In [74]:
for i in movie_info["Distributor"].unique():
    print(i)

롯데컬처웍스(주)롯데엔터테인먼트
(주)넥스트엔터테인먼트월드(NEW)
찬란
씨제이포디플렉스 주식회사,씨지브이 아이스콘
(주)스마일이엔티
유니버설픽쳐스인터내셔널 코리아(유)
한국소니픽쳐스릴리징브에나비스타영화㈜
(주)박수엔터테인먼트
(주)싸이더스
㈜코리아스크린,와이드 릴리즈(주)
(주)영화사 진진
롯데쇼핑㈜롯데엔터테인먼트
(주)제이앤씨미디어그룹
이십세기폭스코리아(주)
(주)프라임엔터테인먼트
(주)영화사 그램,(주)디오시네마
씨제이엔터테인먼트
(주)마노엔터테인먼트
주식회사 마운틴픽쳐스,(주)케이티알파
(주)영화특별시에스엠씨
롯데컬처웍스(주)롯데엔터테인먼트,(주)콘텐츠지오,주식회사 에이투지엔터테인먼트
(주)제이앤씨미디어그룹,와이드 릴리즈(주)
(주)씨제이이엔엠
(주)리틀빅픽쳐스
(주)스튜디오 보난자
(주)콘텐츠지오,(주)키다리스튜디오,㈜아티스트컴퍼니
(주)빌리언스플러스,(주)블루라벨픽쳐스
유한회사 영화사 반딧불
(주)디스테이션,이십세기폭스필름코퍼레이션
예지림엔터테인먼트
(주)영화사오원
(주)인벤트스톤,영화사청어람(주)
이십세기폭스필름코퍼레이션
소니픽쳐스엔터테인먼트코리아주식회사극장배급지점
(주)드림팩트엔터테인먼트
에스케이텔레콤(주)
(주)시네마달
시네마 뉴원
워너브러더스 코리아(주)
(주)디스테이션
(주)마인드마크
(주)나이너스엔터테인먼트
(주)스튜디오 산타클로스엔터테인먼트,(주)제이앤씨미디어그룹
주식회사 플릭스코
(주)쇼박스
(주)에스와이코마드
판씨네마(주)
TCO(주)더콘텐츠온,(주)제이앤씨미디어그룹
(주)프레인글로벌,CGV아트하우스
(주)팝엔터테인먼트
THE 픽쳐스,(주)빅브라더스
롯데컬처웍스(주)롯데시네마,(주)스튜디오디에이치엘,(주)팝엔터테인먼트,이십세기폭스코리아(주)
해피송
(주)토러스엔터테인먼트
스폰지
(주)스튜디오210
(주)싸이더스,스튜디오 에이드
(주)트리플픽쳐스
(주)미디어캐슬,(주)마블엔터테인먼트
(주)인디플러그
(주)키다리이엔티
(주)이놀미디어
TCO(주)더콘텐츠온,(주)디스테이션
브에나비스타인터내셔널코리아
(주)에스와이코마드,글뫼 주식회사
TCO

배급사 여러개가 붙은 게 있다  
감독, 배우를 나눈것처럼 이것도 똑같이 처리한다

In [75]:
distributor = movie_info[["Distributor"]].copy()
# "," 기준으로 분리하여 최대 분할 개수 구하기
max_num = distributor["Distributor"].apply(lambda x: len(str(x).split(','))).max()

# 각 배급사별 컬럼 생성
for i in range(max_num):
    distributor[f"Distributor_{i+1}"] = distributor["Distributor"].apply(
        lambda x: str(x).split(',')[i] if i < len(str(x).split(',')) else None
    )

In [76]:
distributor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4747 entries, 0 to 4768
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Distributor    4747 non-null   object
 1   Distributor_1  4747 non-null   object
 2   Distributor_2  606 non-null    object
 3   Distributor_3  80 non-null     object
 4   Distributor_4  13 non-null     object
 5   Distributor_5  3 non-null      object
 6   Distributor_6  1 non-null      object
 7   Distributor_7  1 non-null      object
 8   Distributor_8  1 non-null      object
dtypes: object(9)
memory usage: 370.9+ KB


In [77]:
temp1 = distributor["Distributor"].str.split(',')
temp2 = []
for i in temp1:
    temp2.extend(i)

pd.set_option('display.max_rows',500)
temp2 = pd.Series(temp2)
temp2.value_counts().reset_index(name="배급영화개수").head(500)

,index,배급영화개수
0,씨제이엔터테인먼트,168
1,유니버설픽쳐스인터내셔널 코리아(유),163
2,(주)영화사 진진,137
3,(주)씨제이이엔엠,135
4,롯데쇼핑㈜롯데엔터테인먼트,129
5,(주)넥스트엔터테인먼트월드(NEW),129
6,(주)쇼박스,125
7,워너브러더스 코리아(주),124
8,(주)디스테이션,116
9,(주)팝엔터테인먼트,106


In [78]:
pd.set_option('display.max_rows',10)

배급사의 종류가 너무 다양해서 과적합이 발생할 것 같다  
일단 저장하자


In [79]:
movie_info = pd.concat([movie_info.drop(columns=["Distributor"]), distributor.drop(columns=["Distributor"])], axis=1)

In [80]:
movie_info.head(2)

,Movie_Title,Main_Country,Grade,Genre,now_showing,director1,director2,director3,director4,director5,...,cgv_genre,movie_summary,Distributor_1,Distributor_2,Distributor_3,Distributor_4,Distributor_5,Distributor_6,Distributor_7,Distributor_8
0,#살아있다,한국,15세이상관람가,드라마,0,조일형,NaN,NaN,NaN,NaN,...,드라마,원인불명 증세의 사람들의 공격에 통제 불능에 빠진 도시. 영문도 모른 채 잠에서 ...,롯데컬처웍스(주)롯데엔터테인먼트,None,None,None,None,None,None,None
1,#아이엠히어,프랑스,12세이상관람가,"멜로/로맨스,코미디",0,에릭 라티고,NaN,NaN,NaN,NaN,...,"코미디, 멜로/로맨스",無대포! 無대책! 無눈치! 프렌치 러버의 좌충우돌 힐링 여행! “나도 지금 전성기...,(주)넥스트엔터테인먼트월드(NEW),None,None,None,None,None,None,None


In [81]:
movie_info.columns

Index(['Movie_Title', 'Main_Country', 'Grade', 'Genre', 'now_showing',
       'director1', 'director2', 'director3', 'director4', 'director5',
       'actor1', 'actor2', 'actor3', 'actor4', 'actor5', 'actor6', 'cgv_genre',
       'movie_summary', 'Distributor_1', 'Distributor_2', 'Distributor_3',
       'Distributor_4', 'Distributor_5', 'Distributor_6', 'Distributor_7',
       'Distributor_8'],
      dtype='object')

### 문제점2 : kofic의 장르(Genre)와 cgv_genre  
kofic의 장르가 더 자세할껄?

In [82]:
movie_info["Genre"].unique()

array(['드라마', '멜로/로맨스,코미디', '다큐멘터리', '공연', '공포(호러)', '액션',
       '액션,어드벤처,범죄,스릴러', '애니메이션', '공포(호러),스릴러', '코미디,드라마', '코미디',
       '스릴러,어드벤처', '액션,드라마,전쟁', '드라마,스릴러', '액션,SF', '드라마,가족',
       '액션,드라마,스릴러', '코미디,멜로/로맨스', '드라마,전쟁', '코미디,판타지', '드라마,코미디',
       '멜로/로맨스,판타지', 'SF,액션,어드벤처', '미스터리,스릴러', '어드벤처,드라마,SF', '범죄,드라마',
       '스릴러', '액션,드라마', '공포(호러),스릴러,드라마', '멜로/로맨스,판타지,드라마', '판타지,드라마',
       '범죄,공포(호러),스릴러', '드라마,멜로/로맨스', '멜로/로맨스,뮤지컬', '멜로/로맨스,드라마', '미스터리',
       'SF,액션,스릴러', '스릴러,드라마', '드라마,미스터리', '전쟁,액션', '드라마,미스터리,스릴러',
       '액션,코미디', '어드벤처,판타지', '범죄,액션,스릴러', '애니메이션,액션,어드벤처', '가족', '드라마,액션',
       '드라마,뮤지컬', '멜로/로맨스', '코미디,드라마,멜로/로맨스', '애니메이션,드라마,판타지',
       '액션,어드벤처,SF', '애니메이션,어드벤처', '드라마,뮤지컬,코미디', '스릴러,미스터리',
       '드라마,멜로/로맨스,미스터리', '드라마,코미디,멜로/로맨스', '코미디,스릴러', '사극', '드라마,코미디,액션',
       'SF', '다큐멘터리,드라마', '범죄,드라마,미스터리', '드라마,어드벤처,액션', '범죄,액션',
       '공연,다큐멘터리', '공포(호러),미스터리,스릴러', '드라마,스릴러,미스터리,범죄,가족',
       '코미디,애니메이션,다큐멘터리', '드라마,범죄', '코미디,범죄', '액션,스릴러,범죄',
  

In [83]:
movie_info["cgv_genre"].unique()

array(['드라마', '코미디, 멜로/로맨스', '다큐멘터리', '공연', '공포(호러)', '액션',
       '액션, 미스터리, 스릴러', '애니메이션', '스릴러', nan, '액션, 드라마, 전쟁',
       '액션, 드라마, 스릴러', '드라마, 전쟁', '코미디', '멜로/로맨스, 판타지', '미스터리, 스릴러',
       '드라마, SF, 어드벤처', '드라마, 공포(호러), 스릴러', '드라마, 멜로/로맨스',
       '드라마, 멜로/로맨스, 판타지', '드라마, 판타지', '멜로/로맨스', '액션, 스릴러', '미스터리',
       '드라마, 스릴러', '드라마, 미스터리', '코미디, 스릴러', '액션, 전쟁', '액션, 코미디', '\u200c',
       '액션, 스릴러, 범죄', '액션, 애니메이션, 어드벤처', '기타', '액션, 드라마', '드라마, 뮤지컬',
       '코미디, 드라마', '애니메이션, 드라마', '액션, SF', '코미디, 드라마, 멜로/로맨스', '사극', 'SF',
       '다큐멘터리, 드라마', '다큐멘터리, 공연', '공포(호러), 미스터리, 스릴러', '액션, 범죄',
       '드라마, 가족, 미스터리, 스릴러, 범죄', '액션, 애니메이션', '코미디, 다큐멘터리', '드라마, 범죄',
       '가족, 어드벤처, 판타지', '드라마, 미스터리, 스릴러', '코미디, 미스터리', '공포(호러), 스릴러',
       '드라마, 가족', '애니메이션, 어드벤처', '공포(호러), 미스터리', '액션, 공포(호러), SF',
       '액션, 미스터리, 어드벤처', '액션, 어드벤처', '코미디, 멜로/로맨스, 판타지', '전쟁', '범죄',
       '어드벤처', '코미디, 스릴러, 범죄', '드라마, SF', '액션, 드라마, 범죄', '애니메이션, 판타지',
       '액션, 애니메이션, 코미디', '공포(호러), 판타지', '드라마, 공포(호러

그냥 전부 분리한 다음에 중복은 제거하자

In [84]:
import pandas as pd
import numpy as np

movie_info.reset_index(drop=True, inplace=True)
genre_df = movie_info[['Movie_Title', 'Genre', 'cgv_genre']].copy()

# 1. 두 장르 열을 콤마(,)로 연결하여 합칩니다.
#    - 결측치(NaN)는 .fillna('')를 통해 빈 문자열로 처리합니다.
combined_genres = genre_df['Genre'].fillna('') + ',' + genre_df['cgv_genre'].fillna('')

# 2. 각 영화의 합쳐진 장르 문자열을 처리하여 고유한 장르 리스트로 만듭니다.
combined_genres = combined_genres.str.replace('\u200c', '')  # 제어 문자 제거
unique_genre_lists = combined_genres.str.split(',').apply(
    lambda x: list(dict.fromkeys([s.strip() for s in x if s.strip()]))
)

# 3. 생성된 장르 리스트를 사용하여 새로운 데이터프레임을 만듭니다.
genres_df = pd.DataFrame(unique_genre_lists.tolist())

# 4. 새로운 데이터프레임의 열 이름을 'genre1', 'genre2', ... 형식으로 변경합니다.
genres_df.columns = [f'genre{i+1}' for i in range(genres_df.shape[1])]

# 5. 기존 데이터프레임과 새로 생성된 장르 데이터프레임을 합칩니다.
result_df = pd.concat([genre_df, genres_df], axis=1)

# 결과 확인
result_df.head()

,Movie_Title,Genre,cgv_genre,genre1,genre2,genre3,genre4,genre5,genre6,genre7
0,#살아있다,드라마,드라마,드라마,None,None,None,None,None,None
1,#아이엠히어,"멜로/로맨스,코미디","코미디, 멜로/로맨스",멜로/로맨스,코미디,None,None,None,None,None
2,#위왓치유,다큐멘터리,다큐멘터리,다큐멘터리,None,None,None,None,None,None
3,(여자)아이들 월드투어 [아이돌] 인 시네마,공연,공연,공연,None,None,None,None,None,None
4,0.0MHz,공포(호러),공포(호러),공포(호러),None,None,None,None,None,None


In [85]:
result_df.isna().sum()

Movie_Title       0
Genre             5
cgv_genre        49
genre1            0
genre2         2009
genre3         3723
genre4         4517
genre5         4701
genre6         4739
genre7         4746
dtype: int64

In [86]:
unique_genre_lists.explode().unique()

array(['드라마', '멜로/로맨스', '코미디', '다큐멘터리', '공연', '공포(호러)', '액션', '어드벤처',
       '범죄', '스릴러', '미스터리', '애니메이션', '전쟁', 'SF', '가족', '판타지', '뮤지컬', '기타',
       '사극', '서부극(웨스턴)', '성인물(에로)', '뮤지컬/연극/오페라'], dtype=object)

In [87]:
pd.set_option('display.max_rows', 30)
unique_genre_lists.explode().value_counts().reset_index(name="장르별영화개수").head(30)

,index,장르별영화개수
0,드라마,2490
1,코미디,832
2,액션,826
3,스릴러,729
4,멜로/로맨스,693
5,애니메이션,500
6,다큐멘터리,407
7,미스터리,336
8,공포(호러),325
9,범죄,304


In [88]:
pd.set_option('display.max_rows',10)

과적합 및 의미부여를 위해 장르명이 서부극(웨스턴), 성인물(에로), 뮤지컬/연극/오페라, 기타인 것은 제거한다 (다시 전처리 진행)

In [89]:
import pandas as pd
import numpy as np

movie_info.reset_index(drop=True, inplace=True)
genre_df = movie_info[['Movie_Title', 'Genre', 'cgv_genre']].copy()

# 1. 두 장르 열을 콤마(,)로 연결하여 합칩니다.
#    - 결측치(NaN)는 .fillna('')를 통해 빈 문자열로 처리합니다.
combined_genres = genre_df['Genre'].fillna('') + ',' + genre_df['cgv_genre'].fillna('')

# 2. 각 영화의 합쳐진 장르 문자열을 처리하여 고유한 장르 리스트로 만듭니다.
combined_genres = combined_genres.str.replace('\u200c', '')  # 제어 문자 제거
combined_genres = combined_genres.str.replace('서부극(웨스턴)', '')
combined_genres = combined_genres.str.replace('성인물(에로)', '')
combined_genres = combined_genres.str.replace('뮤지컬/연극/오페라', '')
combined_genres = combined_genres.str.replace('기타', '')

unique_genre_lists = combined_genres.str.split(',').apply(
    lambda x: list(dict.fromkeys([s.strip() for s in x if s.strip()]))
)

# 3. 생성된 장르 리스트를 사용하여 새로운 데이터프레임을 만듭니다.
genres_df = pd.DataFrame(unique_genre_lists.tolist())

# 4. 새로운 데이터프레임의 열 이름을 'genre1', 'genre2', ... 형식으로 변경합니다.
genres_df.columns = [f'genre{i+1}' for i in range(genres_df.shape[1])]

# 5. 기존 데이터프레임과 새로 생성된 장르 데이터프레임을 합칩니다.
result_df = pd.concat([genre_df, genres_df], axis=1)

# 결과 확인
result_df.head()

,Movie_Title,Genre,cgv_genre,genre1,genre2,genre3,genre4,genre5,genre6,genre7
0,#살아있다,드라마,드라마,드라마,None,None,None,None,None,None
1,#아이엠히어,"멜로/로맨스,코미디","코미디, 멜로/로맨스",멜로/로맨스,코미디,None,None,None,None,None
2,#위왓치유,다큐멘터리,다큐멘터리,다큐멘터리,None,None,None,None,None,None
3,(여자)아이들 월드투어 [아이돌] 인 시네마,공연,공연,공연,None,None,None,None,None,None
4,0.0MHz,공포(호러),공포(호러),공포(호러),None,None,None,None,None,None


In [90]:
result_df.isna().sum()

Movie_Title       0
Genre             5
cgv_genre        49
genre1            7
genre2         2064
genre3         3744
genre4         4525
genre5         4702
genre6         4741
genre7         4746
dtype: int64

7개의 영화를 추가로 제거한다 (genre1에 없다면 나머지도 없다는 뜻)

In [91]:
result_df[result_df["genre1"].isna()]

,Movie_Title,Genre,cgv_genre,genre1,genre2,genre3,genre4,genre5,genre6,genre7
301,"구라, 베토벤",기타,기타,None,None,None,None,None,None,None
2215,사형에 이르는 병,NaN,기타,None,None,None,None,None,None,None
2784,아이를 위한 아이,기타,기타,None,None,None,None,None,None,None
3207,오픈 레인지,서부극(웨스턴),서부극(웨스턴),None,None,None,None,None,None,None
3252,외계+인 2부,기타,기타,None,None,None,None,None,None,None
3833,초록밤,기타,기타,None,None,None,None,None,None,None
4278,팟 제너레이션,NaN,기타,None,None,None,None,None,None,None


In [92]:
result_df.dropna(subset=["genre1"], inplace=True)

In [93]:
movie_info = pd.merge(movie_info.drop(columns=["Genre", "cgv_genre"]), result_df.drop(columns=["Genre", "cgv_genre"]), on="Movie_Title", how="right")
movie_info.head(2)

,Movie_Title,Main_Country,Grade,now_showing,director1,director2,director3,director4,director5,actor1,...,Distributor_6,Distributor_7,Distributor_8,genre1,genre2,genre3,genre4,genre5,genre6,genre7
0,#살아있다,한국,15세이상관람가,0,조일형,NaN,NaN,NaN,NaN,박신혜,...,None,None,None,드라마,None,None,None,None,None,None
1,#아이엠히어,프랑스,12세이상관람가,0,에릭 라티고,NaN,NaN,NaN,NaN,빈센트 네메스,...,None,None,None,멜로/로맨스,코미디,None,None,None,None,None


In [94]:
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4740 entries, 0 to 4739
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Movie_Title    4740 non-null   object
 1   Main_Country   4740 non-null   object
 2   Grade          4740 non-null   object
 3   now_showing    4740 non-null   int64 
 4   director1      4517 non-null   object
 5   director2      297 non-null    object
 6   director3      46 non-null     object
 7   director4      17 non-null     object
 8   director5      4 non-null      object
 9   actor1         4501 non-null   object
 10  actor2         4263 non-null   object
 11  actor3         3245 non-null   object
 12  actor4         2281 non-null   object
 13  actor5         464 non-null    object
 14  actor6         13 non-null     object
 15  movie_summary  4740 non-null   object
 16  Distributor_1  4740 non-null   object
 17  Distributor_2  605 non-null    object
 18  Distributor_3  80 non-null  

### 문제점3 : 감독1과 배우1이 없는 것  
이것은 아예 출연진이 없다는 뜻  
또는 감독은 표시안되어있고 배우만 있는 경우도 있다

감독과 배우 둘 다 없는 경우만 제거하자

In [95]:
movie_info.columns

Index(['Movie_Title', 'Main_Country', 'Grade', 'now_showing', 'director1',
       'director2', 'director3', 'director4', 'director5', 'actor1', 'actor2',
       'actor3', 'actor4', 'actor5', 'actor6', 'movie_summary',
       'Distributor_1', 'Distributor_2', 'Distributor_3', 'Distributor_4',
       'Distributor_5', 'Distributor_6', 'Distributor_7', 'Distributor_8',
       'genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7'],
      dtype='object')

In [96]:
movie_info[["director1", "actor1"]].isna().value_counts()

director1  actor1
False      False     4278
           True       239
True       False      223
Name: count, dtype: int64

둘 다 없는 경우는 없다

### box_office 데이터 처리  
merge를 이용하여 최종 영화 목록으로 필터링한다

In [97]:
box_office = pd.merge(box_office, movie_info[["Movie_Title"]], on="Movie_Title", how="right")
box_office["Movie_Title"].nunique()

4740

# 4. 저장

In [98]:
box_office.to_csv("boxoffice_data.csv", index=False, encoding='utf-8-sig')
movie_info.to_csv("movie_info_data.csv", index=False, encoding='utf-8-sig')

# 5. 추가 수정

박스오피스 데이터에 연도를 추가한다 (미묘한 영화값 상승을 반영)

In [2]:
import pandas as pd
box_office = pd.read_csv("boxoffice_data.csv", encoding='utf-8-sig')
box_office.head(1)

,Movie_Title,Release_Date,Audience_Count,Show_Count,Week,Date,Holiday,Moonhwa,Pandemic,now_showing
0,#살아있다,2020-06-24,204066,7223,1,2020-06-24,0,1,1,0


In [3]:
box_office['Year'] = pd.to_datetime(box_office['Date']).dt.year

In [6]:
box_office.isna().sum()

Movie_Title       0
Release_Date      0
Audience_Count    0
Show_Count        0
Week              0
Date              0
Holiday           0
Moonhwa           0
Pandemic          0
now_showing       0
Year              0
dtype: int64

In [7]:
box_office.to_csv("boxoffice_data.csv", index=False, encoding='utf-8-sig')